In [ ]:
from diffuser import TrainingConfig, TrajDiffuser

In [ ]:
config = TrainingConfig()
diffuser = TrajDiffuser(config, train_batch_num=1)
diffuser.load_model('./closed_diffusion_fullset_ckpt.pth')

In [ ]:
from open_anything_diffusion.datasets.flow_trajectory import FlowTrajectoryDataModule
datamodule = FlowTrajectoryDataModule(
    root="/home/yishu/datasets/partnet-mobility",
    batch_size=1,
    num_workers=30,
    n_proc=2,
    seed=42,
    trajectory_len=config.traj_len,  # Only used when training trajectory model
    special_req="fully-closed",
    # toy_dataset = {
    #     "id": "door-1",
    #     "train-train": ["8994", "9035"],
    #     "train-test": ["8994", "9035"],
    #     "test": ["8867"],
    #     # "train-train": ["8867"],
    #     # "train-test": ["8867"],
    #     # "test": ["8867"],
    # }
    # toy_dataset = {
    #     "id": "door-full-new",
    #     "train-train": ["8877", "8893", "8897", "8903", "8919", "8930", "8936", "8961", "8997", "9016", "9032", "9035", "9041", "9065", "9070", "9107", "9117", "9127", "9128", "9148", "9164", "9168", "9277", "9280", "9281", "9288", "9386", "9388", "9410"],
    #     "train-test": ["8867", "8983", "8994", "9003", "9263", "9393"],
    #     "test": ["8867", "8983", "8994", "9003", "9263", "9393"],
    # }

    # toy_dataset = {
    #     "id": "door-single-test-8867",
    #     "train-train": ["8867"],
    #     "train-test": ["8867"],
    #     "test": ["8867"],
    # }
    toy_dataset = {
        "id": "door-full-new",
        "train-train": ["8877", "8893", "8897", "8903", "8919", "8930", "8961", "8997", "9016", "9032", "9035", "9041", "9065", "9070", "9107", "9117", "9127", "9128", "9148", "9164", "9168", "9277", "9280", "9281", "9288", "9386", "9388", "9410"],
        "train-test": ["8867", "8983", "8994", "9003", "9263", "9393"],
        "test": ["8867", "8983", "8994", "9003", "9263", "9393"],
    }
)

train_val_dataloader = datamodule.train_val_dataloader()
val_dataloader = datamodule.val_dataloader()

In [ ]:
samples = list(enumerate(train_val_dataloader))
sample = samples[23][1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting tools

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(sample.pos.cpu().numpy()[:, 0], sample.pos.cpu().numpy()[:, 1], sample.pos.cpu().numpy()[:, 2])
plt.show()

In [ ]:
# for i in range(1200):
#     sample.pos[i][0] = -1 * sample.pos[i][0]
#     sample.pos[i][1] = -1 * sample.pos[i][1]

#     sample.delta[i][0][0] = -1 * sample.delta[i][0][0]
#     sample.delta[i][0][1] = -1 * sample.delta[i][0][1]

In [ ]:
metrics = []

import tqdm
for i in tqdm.tqdm(range(20)):
    samples = list(enumerate(val_dataloader))
    sample = samples[0][1]
    metric = diffuser.predict([sample], vis=True)
    metrics.append(metric)
    if metric['cos_dist'] > 0.7:
        print("hi")

In [ ]:
metrics

In [ ]:
minus = []
for i in range(20):
    if metrics[i]['cos_dist'] > 0.8:
        minus.append(i)
print(minus)

In [ ]:
metrics[2]['animation'].animate().show()

In [ ]:
metrics[2]

In [ ]:
metrics

In [ ]:
metric_two = diffuser.predict([samples[1][1]], vis=True)

In [ ]:
metric_two

## Unseen

In [ ]:
test_dataloader = datamodule.unseen_dataloader()
samples = list(enumerate(test_dataloader))

In [ ]:
metric_unseen = diffuser.predict([samples[0][1]], vis=True)

In [ ]:
metric_unseen

In [ ]:
for i in range(100):
    metric_unseen = diffuser.predict([samples[3][1]], vis=False)
    if metric_unseen['cos_dist'] > 0:
        print("other direction!")
        break

In [ ]:
i